In [1]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-11-13 20:44:24.648122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 20:44:24.801070: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Num GPUs Available:  1


2024-11-13 20:44:26.323870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 20:44:26.476358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 20:44:26.476650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import tensorflow as tf

USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

Using GPU


In [3]:
import tensorflow_datasets as tfds

# Load the ImageNet dataset

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'

# Load the ImageNet Resized dataset (e.g., 32x32)
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

# Split the dataset into training and validation sets
train_dataset = dataset['train']
val_dataset = dataset['validation']

# Print the dataset info
print(info)

/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tfds.core.DatasetInfo(
    name='imagenet_resized',
    full_name='imagenet_resized/64x64/0.1.0',
    description="""
    This dataset consists of the ImageNet dataset resized to fixed size. The images
    here are the ones provided by Chrabaszcz et. al. using the box resize method.
    
    For [downsampled ImageNet](http://image-net.org/download.php) for unsupervised
    learning see `downsampled_imagenet`.
    
    those from the other ImageNet datasets provided by Tensorflow datasets. See the
    original
    [label list](https://github.com/PatrykChrabaszcz/Imagenet32_Scripts/blob/master/map_clsloc.txt),
    and the
    [labels used by this dataset](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image_classification/imagenet_resized_labels.txt).
    Additionally, the original authors 1 index there labels which we convert to 0
    indexed by subtracting one.
    """,
    config_description="""
    Images resized to 64x64
    """,
    homepage='https://patrykc

2024-11-13 20:44:26.762859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 20:44:26.763753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 20:44:26.763902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 20:44:26.763981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), bu

In [4]:
# from keras.applications import VGG16
from VGG16.vgg import VGG
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

alpha = 0.03
temperature = 0.02

# Load the VGG16 model
model = VGG(alpha=alpha, temperature=temperature)


In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

train_dataset = dataset['train']
val_dataset = dataset['validation']


def preprocess(image, label):
    image = tf.image.resize(image, [64, 64])
    image = preprocess_input(image) 
    return image, label

batch_size = 32
train_dataset = (train_dataset
                 .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                 .shuffle(1000)  
                 .batch(batch_size)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = '/home/asko/tensorflow_datasets/downloads/manual/imagenet_resized/64x64'


# Define the image transformations (resize, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),  # Resize images to a size suitable for VGG16
    transforms.CenterCrop(64),  # Crop to the expected input size for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])

# Load the ImageNet dataset
train_dataset = datasets.ImageNet(root=data_dir, split='train', transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Example: Iterate through the train dataset
for images, labels in train_loader:
    print(images.shape)  # Batch of images
    print(labels.shape)  # Corresponding labels
    break


RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in /home/asko/tensorflow_datasets/downloads/manual/imagenet_resized.

In [ ]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
model.to(device)
print(device)

cuda


In [ ]:
import torch.optim as optim
import torch
optimizer = optim.Adam(model.parameters(), lr=0.001)
torch.set_grad_enabled(True)
# Training Loop
num_epochs = 15
batch_size = 32  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(train_dataset):
        images = torch.from_numpy(images.numpy()).to(device)
        labels = torch.from_numpy(labels.numpy()).to(device)
        output = model(images.reshape(32, 3, 64, 64))
        

        # Calculate risk (combined loss)
        loss = model.risk(Y=labels, output=output)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx > 499:
            break
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_dataset)}")

tensor([866., 880., 115.,  60.,  11.,  60.,  54., 957., 441., 811., 739.,  11.,
         60.,  11.,  60.,  60., 191.,  47.,  47.,  60.,  11., 355., 247.,  11.,
         60., 746., 115., 116., 308., 997., 701.,  11.], device='cuda:0')


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn